In [1]:
import pandas as pd

mini = pd.read_csv('PEC-phimini-ctx-bridge.csv')
# two = pd.read_csv('base-gemma-2b-wikihop.csv')
# nine = pd.read_csv('base-gemma-9b-wikihop.csv')

In [3]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from guidance import models, select

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [4]:
mini.head()

,Unnamed: 0,query,correct,thesis,antithesis,pre-synthesis,context
0,0,The Oberoi family is part of a hotel company t...,delhi,mumbai,The context clearly states that The Oberoi Gr...,The Oberoi family is associated with The Ober...,The Oberoi family is an Indian family that is ...
1,1,Musician and satirist Allie Goertz wrote a son...,president richard nixon,president abraham lincoln,The context clearly states that Milhouse Muss...,The correct answer is 'President Richard Nixo...,She is co-host of Everything's Coming Up Podca...
2,2,What nationality was James Henry Miller's wife?,american,british,"The context states that James Henry Miller, a...",The context clearly states that James Henry M...,James Henry Miller (25 January 1915 – 22 Octob...
3,3,Cadmium Chloride is slightly soluble in this c...,alcohol,sodium chloride,The context states that Cadmium Chloride is s...,The context clearly states that Cadmium Chlor...,It is a hygroscopic solid that is highly solub...
4,4,Which genus of moth in the world's seventh-lar...,crambidae,lepidoptera,The context states that Indogrammodes is a ge...,The context provided mentions that Indogrammo...,India's Andaman and Nicobar Islands share a ma...


In [5]:
from langchain_core.prompts import PromptTemplate
from operator import itemgetter

# prompt augmentation for the (format of the) synthesis:
prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
Choose the most proper option between {options} that best matches with the suggestion. 

Question: {question}
Context: {critique}
Sources: {context}

Assistant:
"""
)
augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }
synthesis_chain = augmentation | prompt_template 

In [6]:
import ast
import re

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

def synthesisGeneration(query, merged, pre_answer, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'critique': pre_answer,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select(merged)
    return str(ans)

def extract_answer_synthesis(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("\n\nAssistant:\n")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("\n\nAssistant:\n")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", use_fast=False)
new_model = models.Transformers(model, tokenizer, temperature=0.0)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/guidance/chat.py:73: UserWarning: Chat template {% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %} was unable to be loaded directly into guidance.
                        Defaulting to the ChatML format which may not be optimal for the selected model. 
                        For best results, create and pass in a `guidance.ChatTemplate` subclass for your model.
  warnings.warn(f"""Chat template {chat_template} was unable to be loaded directly into guidance.


In [ ]:
df = pd.read_csv('hotpot-bridge-updated.csv')

# select a subset of the queries, just for test:
first_queries = df['question']

# same for correct answers and distractors:
correct_answers = df['correct']

# and for the sources:
sources = df['support']

def clean_alternative_column(df):
    # Assicurati che i valori siano stringhe e rimuovi gli spazi bianchi
    df['alternative'] = df['alternative'].apply(lambda x: str(x).replace(' ', '') if isinstance(x, str) else x)
    return df
    
df = clean_alternative_column(df)

N_rows = len(df)

possibilities = []
for i in range(N_rows):
    possibilities.append(str([correct_answers[i], df['alternative'][i]]))

In [ ]:
df = pd.read_csv('wikihop_dataset/wikihop-merged-summarized-test.csv')

# select a subset of the queries, just for test:
first_queries = df['query']

# same for correct answers and distractors:
correct_answers = df['answer']
possibilities = df['options']

# and for the sources:
sources = df['sum_supports']

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/saracandu/my_wikihop/train.csv")

# select a subset of the queries, just for test:
first_queries = df['query']

# same for correct answers and distractors:
correct_answers = df['answer']
possibilities = df['options']

# and for the sources:
sources = df['supports']

In [8]:
import datasets
from datasets import load_dataset

dataset = load_dataset('saracandu/hotpotQA_nli', split="train", trust_remote_code=True)

# select a subset of the queries, just for test:
first_queries = dataset['question']

# same for correct answers and distractors:
correct_answers = dataset['answer']
possibilities = dataset['options']

# and for the sources:
sources = dataset['passages']

N_rows = len(dataset)

In [7]:
import pandas as pd 

df = pd.read_csv('ultramega-test.csv')

# select a subset of the queries, just for test:
first_queries = df['question']

# same for correct answers and distractors:
correct_answers = df['answer']
possibilities = df['options']

# and for the sources:
sources = df['new']

N_rows = len(df)

In [17]:
syn_answers = []
for i in range(len(df)):
    syn_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            mini['pre-synthesis'][i], sources[i])))

In [18]:
ant_answers = []
for i in range(len(df)):
    ant_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            mini['antithesis'][i], sources[i])))

In [8]:
cot_answers = []
for i in range(len(df)):
    cot_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            mini['cot'][i], sources[i])))

In [19]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'thesis': mini['thesis'],
    'pre-antithesis': mini['antithesis'],
    'antithesis': ant_answers,
    'pre-synthesis': mini['pre-synthesis'],
#    'cot': mini['cot'],
    'synthesis': syn_answers,
    'context': sources
} 

df = pd.DataFrame(df)

In [9]:
df = {
    'query': first_queries,
    'correct': correct_answers,
#    'thesis': mini['thesis'],
#    'pre-antithesis': mini['antithesis'],
#    'antithesis': ant_answers,
#    'pre-synthesis': mini['pre-synthesis'],
    'cot': mini['cot'],
    'synthesis': cot_answers,
    'context': sources
} 

df = pd.DataFrame(df)

In [10]:
def clean_text_final(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

In [20]:
# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text_final)
df['thesis'] = df['thesis'].apply(clean_text_final)
df['antithesis'] = df['antithesis'].apply(clean_text_final)
df['synthesis'] = df['synthesis'].apply(clean_text_final)

In [11]:
# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text_final)
# df['thesis'] = df['thesis'].apply(clean_text_final)
# df['antithesis'] = df['antithesis'].apply(clean_text_final)
df['synthesis'] = df['synthesis'].apply(clean_text_final)

In [12]:
df.head()

,query,correct,cot,synthesis,context
0,"Which magazine was started first, Arthur's Mag...",arthurs magazine,Let's analyze the options and the context pro...,arthurs magazine,Arthur's Magazine (1844–1846) was an American ...
1,Which tennis player won more Grand Slam titles...,jonathan stark,"To answer this question, we need to compare t...",jonathan stark,Henri Leconte (born 4 July 1963) is a former F...
2,"Which band was founded first, Hole (the rock b...",the wolfhounds,Assistant: The context provides information a...,the wolfhounds,The Wolfhounds are an indie pop/noise pop band...
3,Were Pavel Urysohn and Leonid Levin known for ...,no,Assistant: The context provides information a...,no,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,Are both The New Pornographers and Kings of Le...,yes,Let's analyze the options and the context pro...,no,Kings of Leon is an American rock band that fo...


In [13]:
df.to_csv('PEC-phimini-purecot-hotpotqa.csv')